# Lab TAT — Root Cause & Bottleneck Analysis


This notebook deepens the baseline EDA by:
- Stage contribution analysis across tests/benches/shifts
- Control charts (daily median TAT with ±3σ bands)
- Outlier (95th percentile) analysis and cross-tabs
- Hourly load vs analytical delay
- STAT vs Routine fairness checks
- Instrument/bench stability indicators
- Rolling trends and simple time trend modeling


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 50)
pd.set_option("display.width", 140)

df = pd.read_csv("../data/lab_events.csv", parse_dates=[
    "collection_time","receipt_time","start_analysis_time","verification_time","report_time"
])

df["tat_pre"]  = (df["receipt_time"] - df["collection_time"]).dt.total_seconds()/60
df["tat_anal"] = (df["verification_time"] - df["start_analysis_time"]).dt.total_seconds()/60
df["tat_post"] = (df["report_time"] - df["verification_time"]).dt.total_seconds()/60
df["tat_total"]= (df["report_time"] - df["collection_time"]).dt.total_seconds()/60
df = df[(df["tat_total"]>=0) & (df["tat_total"]<=7*24*60)]

sla_map = {"CBC":120, "CMP":240, "PT/INR":60, "UA":120, "PathReview":2880}
df["sla_min"] = df["test_code"].map(sla_map).fillna(240)
df["sla_hit"] = (df["tat_total"] <= df["sla_min"]).astype(int)

df["date"] = df["receipt_time"].dt.date
df["hour"] = df["receipt_time"].dt.hour

df.head()


## Stage Contributions (Pre / Anal / Post) by Test / Bench / Shift

In [ ]:

stage_share = (df[["tat_pre","tat_anal","tat_post"]].div(df["tat_total"], axis=0)
               .replace([np.inf,-np.inf], np.nan).fillna(0))
share_df = pd.concat([df[["test_code","bench","shift"]], stage_share], axis=1)

by_test_share = share_df.groupby("test_code")[["tat_pre","tat_anal","tat_post"]].median().sort_values("tat_anal", ascending=False)
by_bench_share = share_df.groupby("bench")[["tat_pre","tat_anal","tat_post"]].median().sort_values("tat_anal", ascending=False)
by_shift_share = share_df.groupby("shift")[["tat_pre","tat_anal","tat_post"]].median().sort_values("tat_anal", ascending=False)

by_test_share, by_bench_share, by_shift_share


In [ ]:

def stacked_bar(df_in, title):
    labels = df_in.index.astype(str)
    pre = df_in["tat_pre"].values
    anal = df_in["tat_anal"].values
    post = df_in["tat_post"].values
    x = np.arange(len(labels))
    fig, ax = plt.subplots(figsize=(10,4))
    ax.bar(x, pre, label="Pre")
    ax.bar(x, anal, bottom=pre, label="Analytical")
    ax.bar(x, post, bottom=pre+anal, label="Post")
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45, ha="right")
    ax.set_ylabel("Share of total TAT")
    ax.set_title(title)
    ax.legend()
    plt.tight_layout()
    plt.show()

stacked_bar(by_test_share, "Median Stage Contributions by Test")
stacked_bar(by_bench_share, "Median Stage Contributions by Bench")
stacked_bar(by_shift_share, "Median Stage Contributions by Shift")


## Control Charts — Daily Median TAT with ±3σ Bands

In [ ]:

daily = df.groupby("date").agg(tat_median=("tat_total","median")).reset_index()
mu = daily["tat_median"].mean()
sigma = daily["tat_median"].std()
ucl = mu + 3*sigma
lcl = max(mu - 3*sigma, 0)

fig, ax = plt.subplots(figsize=(10,4))
ax.plot(daily["date"], daily["tat_median"], marker="o")
ax.axhline(mu, linestyle="--")
ax.axhline(ucl, linestyle="--")
ax.axhline(lcl, linestyle="--")
ax.set_title("Daily Median TAT — Control Chart")
ax.set_ylabel("Minutes")
ax.set_xlabel("Date")
ax.grid(True)
plt.tight_layout()
plt.show()

daily[(daily["tat_median"]>ucl) | (daily["tat_median"]<lcl)].head()


## Outliers (95th Percentile) & Cross-Tabs

In [ ]:

p95 = np.percentile(df["tat_total"], 95)
outliers = df[df["tat_total"] >= p95]

ct_test = outliers["test_code"].value_counts(normalize=True).rename("share_outliers")
ct_shift = outliers["shift"].value_counts(normalize=True).rename("share_outliers")
ct_bench = outliers["bench"].value_counts(normalize=True).rename("share_outliers")

ct_test, ct_shift, ct_bench


## Hourly Load vs Analytical Delay

In [ ]:

by_hour = df.groupby("hour").agg(
    volume=("order_id","count"),
    anal_median=("tat_anal","median"),
    total_median=("tat_total","median")
).reset_index()

fig, ax = plt.subplots(figsize=(10,4))
ax.plot(by_hour["hour"], by_hour["volume"], marker="o")
ax.set_xlabel("Hour of Receipt")
ax.set_ylabel("Volume")
ax.set_title("Hourly Volume")
ax.grid(True)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(10,4))
ax.plot(by_hour["hour"], by_hour["anal_median"], marker="o")
ax.set_xlabel("Hour of Receipt")
ax.set_ylabel("Analytical TAT (median, min)")
ax.set_title("Hourly Analytical Median TAT")
ax.grid(True)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(10,4))
ax.plot(by_hour["hour"], by_hour["total_median"], marker="o")
ax.set_xlabel("Hour of Receipt")
ax.set_ylabel("Total TAT (median, min)")
ax.set_title("Hourly Total Median TAT")
ax.grid(True)
plt.tight_layout()
plt.show()


## STAT vs Routine Fairness Check

In [ ]:

stat_df = df[df["priority"]=="STAT"]
routine_df = df[df["priority"]=="Routine"]

summary = pd.DataFrame({
    "Median_TAT_STAT":[stat_df["tat_total"].median()],
    "Median_TAT_Routine":[routine_df["tat_total"].median()],
    "Median_Pre_STAT":[stat_df["tat_pre"].median()],
    "Median_Pre_Routine":[routine_df["tat_pre"].median()],
})

daily_share_stat = df.groupby("date").apply(lambda x: (x["priority"]=="STAT").mean()).rename("share_stat").reset_index()
daily_routine_tat = routine_df.groupby("date").agg(routine_median=("tat_total","median")).reset_index()
fair = pd.merge(daily_share_stat, daily_routine_tat, on="date", how="inner")

fig, ax = plt.subplots(figsize=(10,4))
ax.scatter(fair["share_stat"], fair["routine_median"])
ax.set_xlabel("STAT Share (by day)")
ax.set_ylabel("Routine Median TAT (min)")
ax.set_title("Does STAT Volume Push Routine TAT Up?")
ax.grid(True)
plt.tight_layout()
plt.show()

summary, fair.head()


## Instrument Stability (Cancellation/Recollect Proxies)

In [ ]:

inst = df.groupby("instrument_id").agg(
    n=("order_id","count"),
    tat_median=("tat_total","median"),
    cancel_rate=("canceled","mean"),
    recollect_rate=("recollect_flag","mean")
).reset_index().sort_values("tat_median", ascending=False)

fig, ax = plt.subplots(figsize=(8,4))
ax.bar(inst["instrument_id"], inst["tat_median"])
ax.set_xlabel("Instrument")
ax.set_ylabel("Median Total TAT (min)")
ax.set_title("Median TAT by Instrument")
plt.tight_layout()
plt.show()

inst.head(10)


## Rolling Trends & Time Trend Modeling

In [ ]:

daily2 = df.groupby("date").agg(
    tat_median=("tat_total","median"),
    sla=("sla_hit","mean")
).reset_index()

daily2["tat_med_roll7"] = daily2["tat_median"].rolling(7, min_periods=3).median()
daily2["sla_roll7"] = daily2["sla"].rolling(7, min_periods=3).mean()

fig, ax = plt.subplots(figsize=(10,4))
ax.plot(daily2["date"], daily2["tat_median"], marker="o", label="Daily Median")
ax.plot(daily2["date"], daily2["tat_med_roll7"], label="7-day Rolling Median")
ax.set_title("Daily Median TAT with Rolling Trend")
ax.set_ylabel("Minutes")
ax.set_xlabel("Date")
ax.grid(True)
ax.legend()
plt.tight_layout()
plt.show()

x = np.arange(len(daily2))
y = daily2["tat_median"].values
coef = np.polyfit(x, y, 1)
trend = coef[0]*x + coef[1]

fig, ax = plt.subplots(figsize=(10,4))
ax.plot(daily2["date"], y, marker="o", label="Daily Median")
ax.plot(daily2["date"], trend, label="Linear Trend")
ax.set_title("Time Trend in Median TAT (Linear Fit)")
ax.set_ylabel("Minutes")
ax.set_xlabel("Date")
ax.grid(True)
ax.legend()
plt.tight_layout()
plt.show()

coef



## Recommendations Scaffold (fill after reviewing outputs)
- Staffing: e.g., add 1 FTE to Evening shift at Bench X → expect median TAT ↓ Y% (based on hourly volume and analytical delay).
- Process: reduce pre-analytical delays between collection and receipt for Test Z (e.g., courier schedule adjustment).
- Quality: investigate Instrument(s) with higher cancellation/recollect rates.
- SLA: test alternative thresholds in the dashboard to balance priorities.

Export a short list of action items with metrics before/after (if you run a simulation in a separate notebook).
